## Build the Neural Network

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

Using mps device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8], device='mps:0')


In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLu: {hidden1} \n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLu: {hidden1}")

Before ReLu: tensor([[ 0.1829,  0.3096, -0.0689,  0.2247,  0.0664, -0.2900,  0.1379, -0.4245,
         -0.2758,  0.4394,  0.0987,  0.3707, -0.3484,  0.6608,  0.2909, -0.4777,
         -0.0122,  0.1574,  0.0908,  0.0359],
        [-0.2395,  0.1966, -0.0473,  0.0955, -0.1122, -0.1504,  0.0863, -0.0640,
         -0.1529,  0.3159,  0.0784,  0.4331, -0.0763,  0.5786,  0.2951, -0.3991,
         -0.3150,  0.4703,  0.4232,  0.1739],
        [ 0.0665,  0.4426, -0.2575,  0.0959, -0.1206, -0.0851,  0.0904, -0.2173,
         -0.0879,  0.0204,  0.4620,  0.1500, -0.1136,  0.3120,  0.2820, -0.2634,
         -0.2065,  0.3765,  0.6510, -0.0301]], grad_fn=<AddmmBackward0>) 


After ReLu: tensor([[0.1829, 0.3096, 0.0000, 0.2247, 0.0664, 0.0000, 0.1379, 0.0000, 0.0000,
         0.4394, 0.0987, 0.3707, 0.0000, 0.6608, 0.2909, 0.0000, 0.0000, 0.1574,
         0.0908, 0.0359],
        [0.0000, 0.1966, 0.0000, 0.0955, 0.0000, 0.0000, 0.0863, 0.0000, 0.0000,
         0.3159, 0.0784, 0.4331, 0.0000, 0.5786, 0.2

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 20)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model} \n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 




Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0348,  0.0288, -0.0285,  ...,  0.0208, -0.0035, -0.0317],
        [-0.0154,  0.0264, -0.0297,  ...,  0.0312,  0.0065, -0.0042]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([0.0296, 0.0216], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0307,  0.0194, -0.0402,  ..., -0.0348,  0.0342,  0.0382],
        [-0.0327, -0.0154,  0.0236,  ..., -0.0170, -0.0382, -0.0250]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values: tensor([0.0347, 0.0041], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values: tensor([[-0.0386,  0.0425,  0.0281,  ...,  0.0344,  0.0013, -0.0109],
        [-0.0184,  0.0308, -0.0059,  ...,  0.016